In [3]:
from keras.layers import Conv2D, Input,MaxPool2D, Reshape,Activation,Flatten, Dense, Permute
from keras.models import Model, Sequential
import tensorflow as tf
from keras.layers.advanced_activations import PReLU
import numpy as np
from keras.utils import plot_model

Using TensorFlow backend.


In [2]:
def create_Kao_Onet( weight_path = 'models/48net.h5'):
    input = Input(shape = [48,48,3])
    x = Conv2D(32, (3, 3), strides=1, padding='valid', name='conv1')(input)
    x = PReLU(shared_axes=[1,2],name='prelu1')(x)
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    x = Conv2D(64, (3, 3), strides=1, padding='valid', name='conv2')(x)
    x = PReLU(shared_axes=[1,2],name='prelu2')(x)
    x = MaxPool2D(pool_size=3, strides=2)(x)
    x = Conv2D(64, (3, 3), strides=1, padding='valid', name='conv3')(x)
    x = PReLU(shared_axes=[1,2],name='prelu3')(x)
    x = MaxPool2D(pool_size=2)(x)
    x = Conv2D(128, (2, 2), strides=1, padding='valid', name='conv4')(x)
    x = PReLU(shared_axes=[1,2],name='prelu4')(x)
    x = Permute((3,2,1))(x)
    x = Flatten()(x)
    x = Dense(256, name='conv5') (x)
    x = PReLU(name='prelu5')(x)

    classifier = Dense(2, activation='softmax',name='conv6-1')(x)
    bbox_regress = Dense(4,name='conv6-2')(x)
    landmark_regress = Dense(10,name='conv6-3')(x)
    model = Model([input], [classifier, bbox_regress, landmark_regress])
    model.load_weights(weight_path, by_name=True)

    return model

In [3]:
def create_Kao_Rnet (weight_path = 'models/24net.h5'):
    input = Input(shape=[24, 24, 3])  # change this shape to [None,None,3] to enable arbitraty shape input
    x = Conv2D(28, (3, 3), strides=1, padding='valid', name='conv1')(input)
    x = PReLU(shared_axes=[1, 2], name='prelu1')(x)
    x = MaxPool2D(pool_size=3,strides=2, padding='same')(x)

    x = Conv2D(48, (3, 3), strides=1, padding='valid', name='conv2')(x)
    x = PReLU(shared_axes=[1, 2], name='prelu2')(x)
    x = MaxPool2D(pool_size=3, strides=2)(x)

    x = Conv2D(64, (2, 2), strides=1, padding='valid', name='conv3')(x)
    x = PReLU(shared_axes=[1, 2], name='prelu3')(x)
    x = Permute((3, 2, 1))(x)
    x = Flatten()(x)
    x = Dense(128, name='conv4')(x)
    x = PReLU( name='prelu4')(x)
    classifier = Dense(2, activation='softmax', name='conv5-1')(x)
    bbox_regress = Dense(4, name='conv5-2')(x)
    model = Model([input], [classifier, bbox_regress])
    model.load_weights(weight_path, by_name=True)
    return model

In [4]:
def create_Kao_Pnet( weight_path = 'models/12net.h5'):
    input = Input(shape=[None, None, 3])
    x = Conv2D(10, (3, 3), strides=1, padding='valid', name='conv1')(input)
    x = PReLU(shared_axes=[1,2],name='PReLU1')(x)
    x = MaxPool2D(pool_size=2)(x)
    x = Conv2D(16, (3, 3), strides=1, padding='valid', name='conv2')(x)
    x = PReLU(shared_axes=[1,2],name='PReLU2')(x)
    x = Conv2D(32, (3, 3), strides=1, padding='valid', name='conv3')(x)
    x = PReLU(shared_axes=[1,2],name='PReLU3')(x)
    classifier = Conv2D(2, (1, 1), activation='softmax', name='conv4-1')(x)
    bbox_regress = Conv2D(4, (1, 1), name='conv4-2')(x)
    model = Model([input], [classifier, bbox_regress])
    model.load_weights(weight_path, by_name=True)
    return model

# Load the models

In [5]:
model_12net = create_Kao_Pnet()
model_24net = create_Kao_Rnet()
model_48net = create_Kao_Onet()

# Plot the models

In [6]:
plot_model(model_12net, to_file='model_24net.png')
model_24net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 24, 24, 3)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 22, 22, 28)   784         input_2[0][0]                    
__________________________________________________________________________________________________
prelu1 (PReLU)                  (None, 22, 22, 28)   28          conv1[0][0]                      
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 11, 11, 28)   0           prelu1[0][0]                     
__________________________________________________________________________________________________
conv2 (Con

# Get the weights with kernel 2x2

In [7]:
def transfer_weights(src_model):
    
    for idx,layer in enumerate(src_model.layers):
        if(layer.name == 'conv3'):
            original_weights = layer.get_weights()[0]
            original_biases = layer.get_weights()[1]
    return original_weights, original_biases

In [8]:
original_weights, original_biases = transfer_weights(model_24net)

print(original_weights.shape)

(2, 2, 48, 64)


# Padd them with zeros

In [9]:
def zero_padding(original_weights, VERBOSE=False):
    # get the shape
    f_w, f_h, f_d, f_num = original_weights.shape
    
    if(VERBOSE):
        print("f_w: {}, f__h: {}, f_d: {}, f_num: {} " .format(f_w, f_h, f_d, f_num))
        print("")
        
    # change te order, just for visualization
    weights = np.transpose(original_weights, (3,2,1,0))
    if(VERBOSE):
        print("weights")
        print(weights)
        print("")

    # create the 0 padding, forst for width, then for height 
    # draw on a paper to understand
    zeros_width = np.zeros((f_num,f_d,1,f_w))
    zeros_height = np.zeros((f_num,f_d,3,1))
    
    
    weights_vertical_padded = np.concatenate((weights, zeros_width), axis=2)
    weights_horizontal_padded = np.concatenate((weights_vertical_padded, zeros_height), axis=3)
    
    modified_weights = np.transpose(weights_horizontal_padded, (3,2,1,0))
    
    return modified_weights

In [10]:
modified_weights = zero_padding(original_weights, False)

# Replace the layer

In [54]:
def new_create_Kao_Rnet (weight_path = 'models/24net.h5'):
    input = Input(shape=[24, 24, 3])  # change this shape to [None,None,3] to enable arbitraty shape input
    x = Conv2D(28, (3, 3), strides=1, padding='valid', name='conv1')(input)
    
    
    
    
    classifier = Dense(2, activation='softmax', name='conv5-1')(x)
    bbox_regress = Dense(4, name='conv5-2')(x)
    
    model = Model([input], [classifier, bbox_regress])
    
    ancient_weights = model.layers[1].get_weights()[0]
    ancient_biases = model.layers[1].get_weights()[1]
    
    print("Before replacing the weights")
    print("ancient_weights.shape: ",ancient_weights.shape)
    print("ancient_biases.shape: ",ancient_biases.shape)
    print("")
    print("ancient_weights")
    print(ancient_weights)
    print("")
    print("ancient_biases")
    print(ancient_biases)
    print("")
    
    # We generate some random weights to replace the original ones
    weights = np.random.random((3,3,3,28))
    biases = np.zeros((28,))
    
    # we add the weights and biases in one list
    weights_biases = []
    weights_biases.append(weights)
    weights_biases.append(biases)
    
    # we replace
    model.layers[1].set_weights(weights_biases)
    
    
    print("After replacing the weights")
    new_weights = model.layers[1].get_weights()[0]
    new_biases = model.layers[1].get_weights()[1]
    
    print("Before replacing the weights")
    print("new_weights.shape: ",new_weights.shape)
    print("new_biases.shape: ",new_biases.shape)
    print("")
    print("new_weights")
    print(new_weights)
    print("")
    print("new_biases")
    print(new_biases)
    print("")
    
    #for idx,layer in enumerate(model.layers):
        #if(layer.name == 'conv1'):
            #model.layers[i].set_weights(weights) 
    
    ''''
    x = PReLU(shared_axes=[1, 2], name='prelu1')(x)
    x = MaxPool2D(pool_size=3,strides=2, padding='same')(x)

    x = Conv2D(48, (3, 3), strides=1, padding='valid', name='conv2')(x)
    x = PReLU(shared_axes=[1, 2], name='prelu2')(x)
    x = MaxPool2D(pool_size=3, strides=2)(x)

    x = Conv2D(64, (2, 2), strides=1, padding='valid', name='conv3')(x)
    x = PReLU(shared_axes=[1, 2], name='prelu3')(x)
    x = Permute((3, 2, 1))(x)
    x = Flatten()(x)
    
    x = Dense(128, name='conv4')(x)
    x = PReLU( name='prelu4')(x)
    
    classifier = Dense(2, activation='softmax', name='conv5-1')(x)
    bbox_regress = Dense(4, name='conv5-2')(x)
    
    model = Model([input], [classifier, bbox_regress])
    model.load_weights(weight_path, by_name=True)
    
    for idx,layer in enumerate(model.layers):
        if(layer.name == 'conv3'):
            original_weights = layer.get_weights()[0]
            original_biases = layer.get_weights()[1]
            
            print("original_weights")
            print(original_weights)
            
            model.layer = []
    '''       
    return model

    
    

In [55]:
new_model_24net = new_create_Kao_Rnet()

Before replacing the weights
ancient_weights.shape:  (3, 3, 3, 28)
ancient_biases.shape:  (28,)

ancient_weights
[[[[-0.11830183 -0.09517246  0.04857461 -0.0110741   0.07691772
     0.14076607 -0.12253118  0.080798   -0.13324134 -0.1461177
    -0.0518246  -0.03283762  0.06046167 -0.13052371  0.14115275
    -0.10770708 -0.12819193  0.01363121  0.07848297 -0.12344369
    -0.05095838  0.12184589  0.05023107 -0.07442015 -0.12363347
     0.01260486 -0.08452534  0.07408376]
   [ 0.01650062  0.09919231  0.03267221  0.08201979  0.06669044
     0.13378306  0.06313889 -0.00488865 -0.00797246  0.14344917
    -0.05811741 -0.04767305  0.02611342 -0.0950589  -0.07135057
     0.08457288 -0.11147858  0.0958682  -0.09106248 -0.05096807
     0.04389088  0.12800933  0.08854109  0.04459155 -0.0478272
     0.06324185 -0.0440629   0.04159707]
   [-0.00526845 -0.06792262  0.04748498  0.00567271 -0.04225403
     0.14220242  0.13280828  0.09083578  0.075752   -0.03383044
    -0.05322604 -0.14612927  0.08246882

In [8]:
def create_A_New_Kao_Rnet():
    input = Input(shape=[24, 24, 3])  # change this shape to [None,None,3] to enable arbitraty shape input
    x = Conv2D(28, (3, 3), strides=1, padding='valid', name='conv1')(input)
    x = PReLU(shared_axes=[1, 2], name='prelu1')(x)
    x = MaxPool2D(pool_size=3,strides=2, padding='same')(x)

    x = Conv2D(48, (3, 3), strides=1, padding='valid', name='conv2')(x)
    x = PReLU(shared_axes=[1, 2], name='prelu2')(x)
    x = MaxPool2D(pool_size=3, strides=2)(x)

    # changed padding to same
    x = Conv2D(64, (3, 3), strides=1, padding='same', name='conv3')(x)
    x = PReLU(shared_axes=[1, 2], name='prelu3')(x)
    # my idea is to use a maxpool
    x = MaxPool2D(pool_size=2, strides=1, name='gaga')(x)
    x = Permute((3, 2, 1))(x)
    x = Flatten()(x)
    x = Dense(128, name='conv4')(x)
    x = PReLU( name='prelu4')(x)
    classifier = Dense(2, activation='softmax', name='conv5-1')(x)
    bbox_regress = Dense(4, name='conv5-2')(x)
    model = Model([input], [classifier, bbox_regress])
    print(model.summary())

In [9]:
create_A_New_Kao_Rnet()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 24, 24, 3)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 22, 22, 28)   784         input_3[0][0]                    
__________________________________________________________________________________________________
prelu1 (PReLU)                  (None, 22, 22, 28)   28          conv1[0][0]                      
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 11, 11, 28)   0           prelu1[0][0]                     
__________________________________________________________________________________________________
conv2 (Con